# **Silver Layer** - Transformação e Limpeza da Tabela Products

In [ ]:

# Notebook baseado na estrutura do professor

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, trim, when, countDistinct
from delta import *

# -------------------------------------------
# 2. Leitura dos dados da camada RAW
# -------------------------------------------
df_products_raw = spark.table("Bronze_Data.product.raw_product_data")

# -------------------------------------------
# 3. Conversão explícita para string + normalização textual (lower + trim)
# -------------------------------------------
for col_name in ["product_id", "cluster_id", "hierarchy1_id", "hierarchy2_id", "hierarchy3_id", "hierarchy4_id", "hierarchy5_id"]:
    df_products_raw = df_products_raw.withColumn(col_name, trim(lower(col(col_name).cast("string"))))

# -------------------------------------------
# 4. Tratamento e limpeza dos dados numéricos
# -------------------------------------------

# Garantir tipo float nas dimensões e substituir negativos e zeros por null
for col_name in ["product_length", "product_depth", "product_width"]:
    df_products_raw = df_products_raw.withColumn(col_name, 
        when(col(col_name).cast("float") > 0, col(col_name).cast("float")).otherwise(None))

# Eliminar linhas com product_id nulo (chave primária)
df_products_clean = df_products_raw.filter(col("product_id").isNotNull())

# Remover duplicados mantendo a última ocorrência
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy("product_id").orderBy(col("product_id"))
df_products_clean = df_products_clean.withColumn("row_num", row_number().over(window_spec))
df_products_clean = df_products_clean.filter(col("row_num") == 1).drop("row_num")

# cluster_id: substituir nulos por "unknown" (mantendo normalização feita antes)
df_products_clean = df_products_clean.withColumn("cluster_id", 
    when(col("cluster_id").isNull(), "unknown").otherwise(col("cluster_id")))

# -------------------------------------------
# 5. Validação de coerência entre hierarquias (cada subnível deve ter apenas 1 pai)
# -------------------------------------------
print("\n📌 Validação de coerência hierárquica:")
for (parent, child) in [("hierarchy1_id", "hierarchy2_id"), ("hierarchy2_id", "hierarchy3_id"), 
                        ("hierarchy3_id", "hierarchy4_id"), ("hierarchy4_id", "hierarchy5_id")]:
    df_check = df_products_clean.groupBy(child).agg(countDistinct(parent).alias("num_pais"))
    incoerencias = df_check.filter(col("num_pais") > 1).count()
    print(f"- {child} associado a múltiplos {parent}: {incoerencias} incoerências")

# -------------------------------------------
# 6. Escrita dos dados limpos na camada SILVER (Delta)
# -------------------------------------------
df_products_clean.write.format("delta").mode("overwrite").saveAsTable("Silver_Data.product.silver_product_data")

# -------------------------------------------
# 8. Visualização final para verificação
# -------------------------------------------
df_products_clean.printSchema()
df_products_clean.show(5)

print("\n✅ Silver Layer - Products concluída com sucesso.")



StatementMeta(, b68c2691-f00f-46e5-b8b2-9baeedc2e0d3, 4, Finished, Available, Finished)


📌 Validação de coerência hierárquica:


- hierarchy2_id associado a múltiplos hierarchy1_id: 0 incoerências


- hierarchy3_id associado a múltiplos hierarchy2_id: 0 incoerências
- hierarchy4_id associado a múltiplos hierarchy3_id: 0 incoerências


- hierarchy5_id associado a múltiplos hierarchy4_id: 0 incoerências


root
 |-- product_id: string (nullable = true)
 |-- product_length: float (nullable = true)
 |-- product_depth: float (nullable = true)
 |-- product_width: float (nullable = true)
 |-- cluster_id: string (nullable = true)
 |-- hierarchy1_id: string (nullable = true)
 |-- hierarchy2_id: string (nullable = true)
 |-- hierarchy3_id: string (nullable = true)
 |-- hierarchy4_id: string (nullable = true)
 |-- hierarchy5_id: string (nullable = true)

+----------+--------------+-------------+-------------+----------+-------------+-------------+-------------+-------------+-------------+
|product_id|product_length|product_depth|product_width|cluster_id|hierarchy1_id|hierarchy2_id|hierarchy3_id|hierarchy4_id|hierarchy5_id|
+----------+--------------+-------------+-------------+----------+-------------+-------------+-------------+-------------+-------------+
|     p0000|           5.0|         20.0|         12.0|   unknown|          h00|        h0004|      h000401|    h00040105|  h0004010534|
|   